In [ ]:
from retrieval_handler.docstore_handler import DocumentStoreHandler
from retrieval_handler.vectorstore_handler import VectorStoreHandler

docstore = DocumentStoreHandler(uri="./docstore", table_name="asian_docstore")
vectorstore = VectorStoreHandler(collection_name="asian_vectorstore", path="./vectorstore")




In [ ]:
from app.containers import Container
container = Container()
container.config.from_yaml("config.yaml")

container.wire(modules=[__name__])
chat_service = container.chat_service()
from uuid import UUID, uuid4
from app.schema import Message

async for content in chat_service.stream_response(id=uuid4(), message=Message(id = str(uuid4()), role="user", content="Xin chào?")):
    print(content, end='', flush=True)


In [ ]:
from uuid import UUID, uuid4
from app.schema import Message

async for content in chat_service.stream_response(id=uuid4(), message=Message(id = str(uuid4()), role="user", content="Việt Nam gia nhập ASEAN vào năm nào?")):
    print(content, end='', flush=True)
